<center>
<img src="https://media.istockphoto.com/id/1399159329/photo/timeline-sound-wave-running-audio-track.jpg?s=612x612&w=0&k=20&c=Ew6W9EPd44G5UWzXk_2lF3RWdl19F4J9iPqUc1XxsmQ=">
    
## [Github](https://github.com/AhmedMansour1070) – Ahmed Hany 
### <center> Email: ahmedhany1070@gmail.com
    
# <center><font  color="red"> Signals Forecasting NoteBook</font>


## Import the packages

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 16, 10
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
import plotly.express as px

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: initialization failed

## <center> Recording An Audio </center>

### <center> The following commented code snipts allows you to record your voice or any audio

In [ ]:
"""frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
"""

In [ ]:
"""stream.stop_stream()
stream.close()
p.terminate()"""

In [ ]:
"""import wave

wf = wave.open("outpuut.wav", "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b"".join(frames))
wf.close()"""

In [ ]:
"""wf = wave.open("output.wav", "rb")
data = wf.readframes(wf.getnframes())
audio = np.frombuffer(data, dtype=np.int16)
audio = audio / np.max(np.abs(audio))
"""

In [ ]:
"""plt.plot(audio, label="true")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show()"""

### Code generates synthetic speech signal with noise.

In [ ]:

# Set sampling rate
fs = 16000

# Generate time axis
t = np.arange(0, 1, 1/fs)

# Set fundamental frequency
f0 = 100

# Generate sine wave
sine_wave = np.sin(2*np.pi*f0*t)

# Add harmonics
for i in range(2, 6):
    sine_wave += np.sin(2*np.pi*f0*i*t) / i

# Add noise
noise = np.random.normal(0, 0.1, len(sine_wave))
speech_signal = sine_wave + noise

# Normalize amplitude to between -1 and 1
speech_signal /= np.max(np.abs(speech_signal))

# Play the signal (assuming you have an audio player installed)
from scipy.io.wavfile import write
write('speech.wav', fs, speech_signal)



In [ ]:
signal_len = speech_signal.shape[0]

In [ ]:
time = np.arange(0 , signal_len/10 , 0.1)
#sin = np.sin(time) + np.random.normal(scale = 0.5 , size = len(time))
sin = speech_signal 

In [ ]:
px.line(sin , title= 'sine function with noise')

In [ ]:
df = pd.DataFrame(dict(sine = sin), index = time , columns = ['sine'])

In [ ]:
df

In [ ]:
train = df.iloc[: int(signal_len*0.8)]
test = df.iloc[int(signal_len*0.8):]
print(len(train) , len(test))

In [ ]:
def create_dataset(X, y, time_steps=1):
    
    
    """
    Works with single features(univariate) and multiple features
 
    """
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)


### Split the data

In [ ]:
time_steps = 10

In [ ]:
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.sine, time_steps)
X_test, y_test = create_dataset(test, test.sine, time_steps)
print(X_train.shape, y_train.shape)


In [ ]:
print(f"X trian : {X_train[0:2]}")
print(f"Y trian : {y_train[0]}")

### Train the model

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(
 units=128,
 input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.Dense(units=1))
model.compile( loss='mean_squared_error',optimizer=keras.optimizers.Adam(0.001))

In [ ]:
history = model.fit(
 X_train, y_train,
 epochs=3,
 batch_size=16,
 validation_split=0.1,
 verbose=1,
 shuffle=False
)

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
reshaped_y_train = np.reshape(y_train, (int(0.8*signal_len) - 10, 1))
reshaped_y_test = np.reshape(y_test, (int(signal_len*0.2) - 10, 1))

data = np.concatenate((reshaped_y_train, reshaped_y_test , y_pred), axis=0)

### Train and Test data

In [ ]:
fig = px.line(data , x = np.arange(0 , len(X_train)) , y =y_train)
fig.add_scatter(x = np.arange(len(y_train) , len(y_train) + len(y_test)), y = y_test, mode='lines', name='Line 2')
fig.add_scatter(x = np.arange(len(y_train) , len(y_train) + len(y_test)), y = y_pred, mode='lines', name='Line 2')

### Train and Test data with the predicted signal plotted with them

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="predicted")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

### Predicted Signal

In [ ]:
px.line(y_pred)

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
window_size = 51
y_pred_flatten = y_pred.flatten()

In [ ]:

window_size = 25
weights = np.ones(window_size)/window_size
mode = 'same'

y_smooth = np.convolve(y_pred_flatten, weights, mode=mode)

print(y_smooth.shape) # Output: (190,)


In [ ]:
plt.plot(y_test, label="true")
plt.plot(y_smooth*1/2, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
y_pred_flat = y_pred.flatten()
sin_flat = sin.flatten()
corr = np.correlate(y_pred_flat, sin_flat[800:990], mode='full')

In [ ]:
px.line(corr)

In [ ]:
import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns

In [ ]:
y_smooth.shape

In [ ]:
y_smooth = y_smooth*3
X = np.fft.fft(y_smooth)

In [ ]:
t = np.arange(0, 1, 1/signal_len)

freqs = np.fft.fftfreq(len(y_smooth), d=t[1]-t[0])

# Plot the original signal and its Fourier transform
fig, axs = plt.subplots(2, 1, figsize=(8, 6))
axs[0].plot(t[int(signal_len*0.8)+ 10:], y_smooth)
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Amplitude')
axs[0].set_title('Smoothed signal')
axs[1].stem(freqs, np.abs(X))
axs[1].set_xlabel('Frequency (Hz)')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlim(-1000, 1000)
axs[1].set_title('Fourier transform')

plt.tight_layout()
plt.show()

In [ ]:
from IPython.display import Audio

# Generate a sine wave signal with a frequency of 440 Hz for 2 seconds
sample_rate = 44100
duration = 2
frequency = 440
#t = np.linspace(0, duration, int(sample_rate * duration), False)
#signal = np.sin(frequency * 2 * np.pi * t)

# Play the signal as audio in Jupyter Notebook
Audio(data=y_smooth, rate=20000)

In [ ]:
!pip install fbprophet
#!pip install prophet

In [ ]:
from fbprophet import Prophet